In [1]:
import netCDF4 as nc
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import collections
import xarray as xr
import statistics
import shapefile
from shapely.geometry import shape, Point
import imageio
import matplotlib.patches as patches
import plotly.express as px
from datetime import datetime
import random
from PIL import Image
from dateutil.relativedelta import relativedelta

In [2]:
#style preferences
plt.style.use('ggplot')
plt.rcParams["font.family"] = "FreeSans"

In [4]:
!pwd

/home/maiampow/code/git/DSSG2023-Groundwater/scripts/analysis


In [5]:
# Load the Jupyter notebook that has the processed GRACE and GLADS data
%run "/home/maiampow/code/git/DSSG2023-Groundwater/notebooks-and-markdowns/04A-Reading & Processing GRACE and GLDAS.ipynb"

Exception: File `'/home/maiampow/code/git/DSSG2023-Groundwater/notebooks-and-markdowns/04A-Reading & Processing GRACE and GLADS.ipynb'` not found.